In [ ]:
#Defin todays URL
url = 'https://files.ssi.dk/Data-Epidemiologiske-Rapport-30102020-pc29'

#Limit dates to plot
from_date='2020-08-1'



In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import requests
import zipfile

In [ ]:
# Direct download data and unpacking
r = requests.get(url, allow_redirects=True)
open('data.zip', 'wb').write(r.content)

with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall("data/")

#define the file to look at
datafolder=Path("data/")
datafile=datafolder / "Test_pos_over_time.csv"

In [ ]:
# this is nice, skips bad lines (which does not convert to date) and converts index to date
#Notice handeling of danish format of the numbers
df=pd.read_csv(datafile,  sep=';', parse_dates=['Date'], index_col=['Date'],error_bad_lines=False, engine='python', skipfooter=2, decimal=',', thousands='.')


In [ ]:
# calculate some relevant numbers

def calcScaledNumber (row):
    if row.NotPrevPos > 0 :
        return row.NewPositive / row.NotPrevPos**0.7
    else:
        return 0
    
    
def calcPosPct (row):
    if row.NotPrevPos > 0 :
        return row.NewPositive / row.NotPrevPos * 100
    else:
        return 0

df['ScaledNumber']=df.apply(lambda row: calcScaledNumber(row), axis=1)
df['PosPct']=df.apply(lambda row: calcPosPct(row), axis=1)


In [ ]:
# define a common title
title_str='SSI COVID-19 data, tilfælde opgjort på prøvetagningsdato'

In [ ]:
df[from_date:].plot(y='NewPositive',title=title_str,style='.')

In [ ]:
ax=df[from_date:].plot(y='PosPct',title=title_str,label='NewPositive / NotPrevPosTested * 100',style='.')
ax.set_ylabel("%")

In [ ]:
df[from_date:].plot(y='ScaledNumber',title=title_str,label='NewPositive / NotPrevPosTested ^ 0.7',style='.')